<a href="https://colab.research.google.com/github/sagawritescode/ENDTwoPointOPhase1/blob/main/END_3_Assignment_Submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision # provide access to datasets, models, transforms, utils, etc
import torchvision.transforms as transforms
from torch.utils.data import Dataset

In [2]:
import random
class Digits(Dataset):
  def __init__(self):
    self.train_set = torchvision.datasets.MNIST(
      root='./data'
      ,train=True
      ,download=True
      ,transform=transforms.Compose([
          transforms.ToTensor()
      ])
    )
  
  def __len__(self):
    return len(self.train_set)

  def __getitem__(self, idx):
    # fetch the element 
    sample = self.train_set[idx]
    image,label = sample 
    # generate a random number 
    random_number = random.randint(0,9)
    # encode the label as one hot vector 
    onehot_label = torch.nn.functional.one_hot(torch.as_tensor(label),10)
    # encode the random_number + predicted image label as one hot vector 
    onehot_expected_sum_label = torch.nn.functional.one_hot(torch.as_tensor(random_number+label),20)
    # concatenate the labels and one_hot_expected_sum into a single tensor 
    new_tensor = torch.cat((onehot_label, onehot_expected_sum_label), 0)
    # reshape the input image to 1d tensor 
    one_d_input_image = image.reshape(28*28)
    # encode the random number to a one hot encoded vector 
    one_hot_random_number = torch.nn.functional.one_hot(torch.as_tensor(random_number),28*28)
    # we concatenate input image and one_hot_random_number 
    concatenated_input = torch.cat((one_d_input_image, one_hot_random_number), 0)
    return concatenated_input,new_tensor

In [4]:
import torch.optim as optim

torch.set_grad_enabled(True) 

In [5]:
import torch.nn as nn 
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        # the same network we used in assignment 1
        self.input = nn.Linear(in_features=784*2, out_features = 784)
        self.fc1 = nn.Linear(in_features=784, out_features=120)        
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=30)

    def forward(self, t):
        t = t
        t = self.input(t)
        t = torch.tanh(t)
        
        # (1) hidden linear layer
        t = self.fc1(t)
        t = torch.tanh(t)

        # (2) hidden linear layer
        t = self.fc2(t)
        t = torch.relu(t)

        # (6) output layer
        t = self.out(t)
        

        return t

In [6]:

def is_correct(pred,label):
  # find the index of maxmimum elements among the first ten bits  
  # if the label has a 1 in that index then the predicted value is correct 
  pred_number_index = pred[0:10].argmax(dim=0)
  pred_sum_index = pred[10:].argmax(axis=0)

  # find the index of maxmimum elements among the last 20 bits  
  # if the label has a 1 in that index then the predicted value is correct    
  pred_actual_sum_index = 10 + pred_sum_index
  label_number_index = label[0:10].argmax(axis=0)
  label_sum_index = label[10:].argmax(axis=0)
  label_actual_sum_index = 10 + label_sum_index
  # a checks if the predicted digit is equal to the actual digit
  a = torch.eq(pred_number_index,label_number_index) 
  # b checks if the predicted sum is equal to the actual sum
  b = torch.eq(pred_actual_sum_index,label_actual_sum_index) 
  # if both are correct then return true 
  return a and b 

def get_num_correct(preds, labels):
  batch_correct_count = 0
  for pred,label in zip(preds,labels):
    if is_correct(pred,label):
      batch_correct_count+=1
  return batch_correct_count


In [7]:
label = torch.as_tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])
pred = torch.as_tensor([ 8.8802e-03, -9.2213e-03,  2.1304e-01,  1.7784e-01, -5.8605e-02,  3.3348e-02,  8.9785e-02, -5.2169e-02,
         1.1994e-01, -1.2858e-01,  1.2525e-01,  6.0575e-02, -2.2036e-01,  9.9685e-02, -2.5213e-02, -4.5135e-03,
         1.0846e-01, -9.4002e-02, -3.7245e-02,  8.7239e-02,  1.2691e-01, -2.4399e-02,  2.8530e-02,  1.9595e-02,
        -3.6636e-02, -1.9829e-04, -3.8571e-02,  2.2719e-02, -2.4126e-02, -2.5132e-02])
is_correct(label,pred)

tensor(False)

In [9]:
network = Network()

train_loader = torch.utils.data.DataLoader(Digits(), batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.000001)

total_loss = 0
total_correct = 0

for batch in train_loader: # Get Batch
    images, labels = batch 
    optimizer.zero_grad()
    preds = network(images)
    loss_func = nn.L1Loss()
    loss = loss_func.forward(preds, labels)
    loss.backward() # Calculate Gradients
    optimizer.step() # Update Weights

    total_loss += loss.item()
    total_correct += get_num_correct(preds, labels)

print(
    "epoch:", 0, 
    "total_correct:", total_correct, 
    "loss:", total_loss
)





Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


epoch: 0 total_correct: 627 loss: 66.1006851196289


In [10]:
for epoch in range(50):

    total_loss = 0
    total_correct = 0

    for batch in train_loader: # Get Batch
        images, labels = batch 
        optimizer.zero_grad(True)
        preds = network(images)
        loss_func = nn.L1Loss()
        loss = loss_func.forward(preds, labels)
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights
        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )

epoch 0 total_correct: 460 loss: 58.086165465414524
epoch 1 total_correct: 505 loss: 53.46803671121597
epoch 2 total_correct: 734 loss: 50.353261172771454
epoch 3 total_correct: 721 loss: 48.47000198811293
epoch 4 total_correct: 852 loss: 47.39283736050129
epoch 5 total_correct: 934 loss: 46.61238111555576
epoch 6 total_correct: 899 loss: 45.97095839679241
epoch 7 total_correct: 907 loss: 45.38799428939819
epoch 8 total_correct: 908 loss: 44.84988898038864
epoch 9 total_correct: 950 loss: 44.36904316395521
epoch 10 total_correct: 948 loss: 43.95465601980686
epoch 11 total_correct: 1041 loss: 43.59113258123398
epoch 12 total_correct: 1060 loss: 43.26929011940956
epoch 13 total_correct: 1097 loss: 42.98249430209398
epoch 14 total_correct: 1027 loss: 42.72978004813194
epoch 15 total_correct: 1110 loss: 42.50060995668173
epoch 16 total_correct: 1207 loss: 42.28969715535641
epoch 17 total_correct: 1255 loss: 42.09183843433857
epoch 18 total_correct: 1390 loss: 41.90082913637161
epoch 19 tot